# Functions du programme Open Loop sans FeedForward

## 1. Import

In [1]:
from signal import signal
from tokenize import Single
import numpy as np
from scipy.optimize import minimize
from matplotlib import colors as mcolors

%matplotlib qt
import matplotlib.pyplot as plt

from matplotlib.widgets import Slider, Button, RadioButtons,TextBox,CheckButtons
import matplotlib.patches as mpatches
import datetime
import os
import tclab
import time

from xmlrpc.client import Boolean
import numpy as np

from IPython.display import display, clear_output

import package_Class
from package_Class import Simulation,Path,FirstOrder,SecondOrderPlusDelay,LeadLag,FeedForward,PID_Controller,Delay,Signal,Variable,Graph,LabValues

## 2. Création Instance Classe simulation 
Va prendre les param commun a toutes les classes

In [2]:
SIM = Simulation(2000,1,26,True,'Simulation_OPL')
#help(SIM)

 [elapsed time: 0:00:00] |                                  | (ETA:  --:--:--)


## 3. Création Instance Classe Graph
Classe servant a l'affichage des valeurs calculées 

In [3]:
G = Graph(SIM,'PID Control_')
#help(G)

## 4. Création Instance Classe Path
Decrit les chemin des signeaux en entrée

In [4]:
SP = Path(SIM,{0: 50, SIM.TSim: 50})
DV = Path(SIM,{0: 50, 800 : 40, 1400 : 60, SIM.TSim: 60})
MAN = Path(SIM,{0: 0, SIM.TSim: 0})
MANV = Path(SIM,{0: 50, SIM.TSim: 50})
#help(SP)

## 5. Création Instance Classe FirstOrder
Prossesus P(s) et D(s) du premier degrée avec leurs paramètres 


In [5]:
P = FirstOrder(SIM,0.6522434279003099,245.9823790885576,0.649693920059717,50,SIM.PVInit)
D = FirstOrder(SIM,0.6156105636473335,387.0591022229922, 5.419428855220769,50,0)
#help(P)

## 6. Création Instance Classe FeedForward
Prend les prossesus P(s) et D(s) en entrée 

In [6]:
FF = FeedForward(SIM,P,D,False)
#help(FF)

## 7. Création Instance Classe PID_Controller + IMC methode
Creation instace PID et execution de la methode IMC

In [7]:
PID = PID_Controller(SIM,0.49,180,10,6,0,100,True,False)
#PID.IMC_tuning(P,0.4,'H')
#help(PID)

## 8. Boucle pour une simulation / Experience
Si la variable SIM.sim de la simulation est vrais.
Class.RT calcule les valeurs du bloc en instantanée

In [8]:
if(SIM.sim == True):
    t = []
    for ti in SIM.t:
        t.append(ti)
        #Signals
        SP.RT(t)
        DV.RT(t)
        MAN.RT(t)
        MANV.RT(t)

        FF.RT(DV.Signal) # FeedForward
        PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,FF.MVFF,'EBD-EBD')

        SIM.addMV(PID.MVFB,FF.MVFF) # Modified Value

        P.RT(SIM.MV,'EBD')
        D.RT(DV.Signal,'EBD')
        SIM.PV.append(P.PV[-1]+D.PV[-1]) # Point Value
        SIM.updateBar()


 [elapsed time: 0:00:01] |                                  | (ETA:  --:--:--)
 [elapsed time: 0:00:01] |████████                          | (ETA:   0:00:03)
 [elapsed time: 0:00:01] |█████████                         | (ETA:   0:00:03)
 [elapsed time: 0:00:01] |█████████████                     | (ETA:   0:00:02)
 [elapsed time: 0:00:01] |█████████████████                 | (ETA:   0:00:01)
 [elapsed time: 0:00:01] |████████████████████              | (ETA:   0:00:01)
 [elapsed time: 0:00:01] |███████████████████████           | (ETA:   0:00:00)
 [elapsed time: 0:00:01] |██████████████████████████        | (ETA:   0:00:00)
 [elapsed time: 0:00:01] |████████████████████████████      | (ETA:   0:00:00)
 [elapsed time: 0:00:01] |██████████████████████████████    | (ETA:   0:00:00)
 [elapsed time: 0:00:01] |█████████████████████████████████ | (ETA:   0:00:00)


Si la variable SIM.sim de la simulation est fausse alors ouvre la connection TcLab 

In [9]:
if(SIM.sim == False):
    #Tc Lab
    LAB = tclab.TCLab()
    LABVal = LabValues(SIM,LAB)

    SIM.t = []
    start = time.time()
    delta = 0
    totalTime = 0
    last = time.time()

    while totalTime < SIM.TSim:
        if delta > SIM.Ts:
            last = time.time()
            SIM.t.append(round(totalTime,4))
            #Signals
            SP.RT(SIM.t)
            DV.RT(SIM.t)
            MAN.RT(SIM.t)
            MANV.RT(SIM.t)

            FF.RT(DV.Signal) # FeedForward
            PID.RT(SP.Signal,SIM.PV,MAN.Signal,MANV.Signal,FF.MVFF,'EBD-EBD')

            SIM.addMV(PID.MVFB,FF.MVFF) # Modified Value

            LABVal.RT(SIM.MV,DV.Signal,D.point_fct)
            delta = 0
            SIM.updateBar()

        else :
            totalTime = time.time() - start
            delta = time.time() - last
    LAB.close()
        

## 9. On cree des liste d'instances Signal que l'on veux afficher 
Il suffit de mettre en commentaire ou rajouter pour les afficher

In [10]:
# Signaux Afficher dans  Graph Binaires
SigValsBin = [
    Signal(MAN.Signal,'Manual Mode','-g')
]

# Signaux Afficher dans  Graph 1 : Temperature
SigVals1 = [
    Signal(SP.Signal,'SP','-r'),
    Signal(SIM.PV,'PV','-b'),
    #Signal(P.PV,'P(s)','--b'),
    #Signal(D.PV,'D(s)','--k'),
]

# Signaux Afficher dans Graph 2 : % de chauffe
SigVals2 = [
    Signal(SIM.MV,'MV','-b'),
    Signal(DV.Signal,'DV','-k'),
    #Signal(MANV.Signal,'MANVal','-m'),
    #Signal(FF.MVFF,'MVFF','-g'),
    #Signal(PID.MVFB,'MVFB','-y'),
    #Signal(PID.E,'E',':r'),
    Signal(PID.MVP,'MVP',':g'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
    #Signal(DV.Signal,'DV','-k'),
]

# Signaux enregistrer dans le .txt
SigSave = [
    Signal(SIM.MV,'MV','-b'),
    Signal(PID.MVP,'MVP',':b'),
    Signal(PID.MVI,'MVI',':y'),
    Signal(PID.MVD,'MVD',':m'),
    Signal(SP.Signal,'SP',':m'),
    Signal(SIM.PV,'PV',':m'),
    Signal(DV.Signal,'DV','-k'),
    Signal(MAN.Signal,'Man','-k'),
]

# Variables affichées sur le graph
varVals = [
    Variable(SIM.TSim,'Temps Sim [s]'),
    Variable(SIM.Ts,'Sampling [s]'),
    Variable(SIM.PVInit,'Pv Init [°C]'),

    Variable(PID.OLP,'Open Loop'),
    Variable(PID.ManFF,'Man FF'),


    Variable(PID.Kc,'Kc PID'),
    Variable(PID.Td,'Td PID'),
    Variable(PID.Ti,'Ti PID'),
    Variable(PID.gamma,'Gamma IMC'),

    Variable(FF.active,'FF Enabled'),
    Variable(FF.T1p,'TLead P(s)'),
    Variable(FF.T2p,'TLag P(s)'),
    Variable(FF.T1d,'TLead D(s)'),
    Variable(FF.T2d,'TLag D(s)'),

]

## 10. Affichage du graph des signaux et bode

Marche pas bien dans kernel -> MathPlotLib Ram

In [11]:
G.show([SigVals1,SigVals2,SigSave],SigValsBin,varVals)

In [12]:
#G.Bode(P,PID,'PID')

In [13]:
#G.Bode(P,PID,'P')